# Demo Notebook for Metrics Correlation Model Testing

#### [Download notebook](https://github.com/opensearch-project/opensearch-py-ml/blob/main/docs/source/examples/demo_metrics_correlation.ipynb)


## Introduction

This notebook introduces the technique of metrics correlation using ML Commons API.

## Step 0: Imports

Please install the following packages from the terminal if you haven't already. They can be also installed from the notebook by uncommenting the line and execute.

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# !pip install pandas matplotlib numpy opensearch-py opensearch-py-ml

In [2]:
# import this to stop opensearch-py-ml from yelling every time a DataFrame connection made
import warnings
warnings.filterwarnings('ignore')

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import opensearch_py_ml as oml
from opensearchpy import OpenSearch

# Import standard test settings for consistent results
from opensearch_py_ml.conftest import *

## Step 1: Set up clients and define helper functions


In [16]:
CLUSTER_URL = 'https://localhost:9200'

def get_os_client(cluster_url = CLUSTER_URL,
                  username='admin',
                  password='admin'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts=[cluster_url],
        http_auth=(username, password),
        verify_certs=False
    )
    return client


In [17]:
client = get_os_client()

## Step 2: Create data index


In [37]:
# Reading csv files as a dataframes
df_pd = pd.read_csv("data/SMD_small.csv.zip", header=None, index_col=None)
Y = pd.read_csv("data/SMD_small_labels.csv.zip", header=None, index_col=None)

There are 100 rows and 38 columns in the dataset. Each row represents a certain time point, while each columns represents a metric. This is a regular time series dataset.

In [38]:
df_pd

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
0,0.096774,0.088983,0.090301,0.102207,0.0,0.943590,0.162900,0.0,0.022140,0.000000,0.151626,0.067568,0.027397,0.057718,0.096116,0.166648,0.0,0.0,0.139421,0.125356,0.121258,0.097189,0.978899,0.014167,0.041280,0.013088,0.0,0.101560,0.0,0.038682,0.090090,0.058672,0.000386,0.000022,0.102150,0.102143,0.0,0.0
1,0.107527,0.062500,0.081382,0.097561,0.0,0.948718,0.164783,0.0,0.021714,0.000244,0.137600,0.067568,0.027397,0.054454,0.096512,0.151092,0.0,0.0,0.136662,0.122707,0.122660,0.098542,0.912582,0.013654,0.043204,0.012626,0.0,0.102762,0.0,0.038682,0.091338,0.061245,0.000000,0.000034,0.103308,0.103301,0.0,0.0
2,0.107527,0.104873,0.094760,0.101045,0.0,0.948718,0.167608,0.0,0.021856,0.000000,0.355348,0.067568,0.013699,0.047754,0.102259,0.369441,0.0,0.0,0.136871,0.124260,0.121560,0.097866,0.969999,0.014744,0.043030,0.013521,0.0,0.101974,0.0,0.038682,0.090794,0.058672,0.000386,0.000011,0.101499,0.101564,0.0,0.0
3,0.096774,0.123941,0.100334,0.103368,0.0,0.938462,0.167608,0.0,0.021289,0.000122,0.213240,0.067568,0.013699,0.045177,0.101268,0.229628,0.0,0.0,0.140062,0.124260,0.121863,0.098005,0.906840,0.013494,0.042505,0.012452,0.0,0.101974,0.0,0.038682,0.090714,0.059187,0.000386,0.000034,0.101354,0.101419,0.0,0.0
4,0.096774,0.070975,0.089186,0.098722,0.0,0.938462,0.167608,0.0,0.024979,0.002211,0.164265,0.108108,0.027397,0.058882,0.107015,0.177700,0.0,0.0,0.138460,0.126014,0.120983,0.096752,0.947678,0.015417,0.040931,0.014099,0.0,0.101422,0.0,0.038682,0.089770,0.061245,0.000386,0.000022,0.101861,0.101926,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.494624,0.072034,0.055741,0.048780,0.0,0.892308,0.177966,0.0,0.007664,0.000000,0.165382,0.013514,0.000000,0.062581,0.031312,0.180576,0.0,0.0,0.067567,0.056099,0.060808,0.046049,0.546401,0.008205,0.018016,0.007627,0.0,0.051302,0.0,0.037249,0.043324,0.038600,0.000000,0.000034,0.052704,0.052773,0.0,0.0
96,0.494624,0.127119,0.074693,0.055749,0.0,0.892308,0.177966,0.0,0.014051,0.000000,0.182568,0.040541,0.013699,0.043924,0.067776,0.197684,0.0,0.0,0.059534,0.053888,0.054018,0.041255,0.503840,0.009968,0.015043,0.009332,0.0,0.045707,0.0,0.037249,0.038859,0.030880,0.000000,0.000034,0.048867,0.048936,0.0,0.0
97,0.494624,0.069915,0.066890,0.055749,0.0,0.892308,0.177024,0.0,0.015612,0.000000,0.142494,0.094595,0.027397,0.045659,0.074118,0.157678,0.0,0.0,0.068750,0.056862,0.060258,0.045770,0.529534,0.009231,0.018016,0.008552,0.0,0.050948,0.0,0.037249,0.043132,0.037571,0.000000,0.000034,0.051546,0.051542,0.0,0.0
98,0.494624,0.046610,0.060201,0.052265,0.0,0.892308,0.177024,0.0,0.013341,0.000000,0.141723,0.067568,0.013699,0.050564,0.061038,0.158170,0.0,0.0,0.068306,0.054441,0.055200,0.041931,0.466016,0.007083,0.015917,0.006645,0.0,0.046397,0.0,0.037249,0.039291,0.028307,0.000000,0.000034,0.047709,0.047705,0.0,0.0


However, to use this dataset for metric correlation later on, each row needs to represent a metric. Therefore, we need to transpose it before populating the index.

In [39]:
df_pd = df_pd.transpose()
df_pd

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.096774,0.107527,0.107527,0.096774,0.096774,0.096774,0.096774,0.096774,0.096774,0.096774,0.096774,0.096774,0.096774,0.096774,0.096774,0.096774,0.096774,0.096774,0.086022,0.086022,0.086022,0.086022,0.086022,0.086022,0.086022,0.086022,0.086022,0.086022,0.086022,0.086022,0.086022,0.086022,0.075269,0.075269,0.086022,0.075269,0.075269,0.075269,0.075269,0.075269,0.075269,0.075269,0.075269,0.064516,0.064516,0.064516,0.075269,0.075269,0.075269,0.064516,0.064516,0.064516,0.064516,0.064516,0.064516,0.064516,0.064516,0.064516,0.064516,0.053763,0.064516,0.064516,0.053763,0.064516,0.064516,0.053763,0.053763,0.053763,0.053763,0.053763,0.053763,0.064516,0.053763,0.053763,0.053763,0.053763,0.053763,0.053763,0.053763,0.053763,0.043011,0.043011,0.053763,0.053763,0.053763,0.086022,0.096774,0.376344,0.494624,0.494624,0.494624,0.494624,0.494624,0.494624,0.494624,0.494624,0.494624,0.494624,0.494624,0.494624
1,0.088983,0.062500,0.104873,0.123941,0.070975,0.077331,0.154661,0.104873,0.077331,0.069915,0.050847,0.096398,0.118644,0.099576,0.127119,0.086864,0.062500,0.093220,0.074153,0.069915,0.148305,0.104873,0.039195,0.051907,0.128178,0.048729,0.045551,0.195975,0.083686,0.110169,0.066737,0.100636,0.073093,0.059322,0.054025,0.059322,0.050847,0.034958,0.052966,0.059322,0.065678,0.083686,0.077331,0.066737,0.082627,0.063559,0.045551,0.050847,0.072034,0.092161,0.075212,0.050847,0.034958,0.036017,0.038136,0.069915,0.041314,0.091102,0.069915,0.047669,0.023305,0.047669,0.070975,0.069915,0.080508,0.055085,0.086864,0.072034,0.028602,0.090042,0.065678,0.077331,0.059322,0.067797,0.046610,0.029661,0.059322,0.056144,0.050847,0.050847,0.029661,0.038136,0.022246,0.037076,0.070975,0.045551,0.100636,0.085805,0.025424,0.013771,0.044492,0.106992,0.079449,0.078390,0.057203,0.072034,0.127119,0.069915,0.046610,0.052966
2,0.090301,0.081382,0.094760,0.100334,0.089186,0.088071,0.102564,0.099220,0.095875,0.089186,0.080268,0.090301,0.095875,0.093645,0.102564,0.095875,0.085842,0.089186,0.085842,0.076923,0.092531,0.086957,0.071349,0.069119,0.085842,0.066890,0.063545,0.098105,0.089186,0.098105,0.086957,0.093645,0.088071,0.081382,0.073579,0.075808,0.069119,0.055741,0.055741,0.056856,0.060201,0.069119,0.074693,0.070234,0.078038,0.071349,0.063545,0.062430,0.064660,0.071349,0.070234,0.064660,0.047938,0.047938,0.045708,0.059086,0.053512,0.070234,0.068004,0.061315,0.051282,0.053512,0.059086,0.056856,0.063545,0.060201,0.068004,0.064660,0.049052,0.060201,0.059086,0.066890,0.063545,0.061315,0.055741,0.049052,0.051282,0.052397,0.051282,0.050167,0.042363,0.042363,0.035674,0.039019,0.051282,0.043478,0.055741,0.055741,0.045708,0.037904,0.040134,0.059086,0.060201,0.059086,0.055741,0.055741,0.074693,0.066890,0.060201,0.059086
3,0.102207,0.097561,0.101045,0.103368,0.098722,0.097561,0.102207,0.101045,0.099884,0.096400,0.092915,0.096400,0.097561,0.096400,0.099884,0.097561,0.092915,0.094077,0.091754,0.087108,0.091754,0.089431,0.083624,0.081301,0.087108,0.078978,0.076655,0.087108,0.084785,0.088269,0.084785,0.087108,0.085947,0.083624,0.080139,0.081301,0.077816,0.072009,0.070848,0.069686,0.069686,0.070848,0.073171,0.072009,0.074332,0.072009,0.069686,0.067364,0.067364,0.069686,0.068525,0.066202,0.059233,0.058072,0.055749,0.060395,0.058072,0.063879,0.062718,0.060395,0.056911,0.056911,0.059233,0.058072,0.059233,0.058072,0.060395,0.059233,0.052265,0.056911,0.056911,0.059233,0.058072,0.056911,0.055749,0.052265,0.052265,0.052265,0.052265,0.052265,0.049942,0.048780,0.045296,0.045296,0.049942,0.045296,0.049942,0.049942,0.046458,0.042973,0.042973,0.049942,0.049942,0.049942,0.048780,0.048780,0.055749,0.055749,0.052265,0.052265
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [40]:
# Creating and populating an index with the dataset
df = oml.pandas_to_opensearch(df_pd,
                    os_client=client,
                     os_dest_index='smd',
                    os_if_exists="replace",
                    os_refresh=True,)

In [41]:
df

,0,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,24,25,26,27,28,29,3,30,31,32,33,34,35,36,37,38,39,4,40,41,42,43,44,45,46,47,48,49,5,50,51,52,53,54,55,56,57,58,59,6,60,61,62,63,64,65,66,67,68,69,7,70,71,72,73,74,75,76,77,78,79,8,80,81,82,83,84,85,86,87,88,89,9,90,91,92,93,94,95,96,97,98,99
0,0.096774,0.107527,0.096774,0.096774,0.096774,0.096774,0.096774,0.096774,0.096774,0.096774,0.086022,0.086022,0.107527,0.086022,0.086022,0.086022,0.086022,0.086022,0.086022,0.086022,0.086022,0.086022,0.086022,0.096774,0.086022,0.086022,0.075269,0.075269,0.086022,0.075269,0.075269,0.075269,0.075269,0.075269,0.096774,0.075269,0.075269,0.075269,0.064516,0.064516,0.064516,0.075269,0.075269,0.075269,0.064516,0.096774,0.064516,0.064516,0.064516,0.064516,0.064516,0.064516,0.064516,0.064516,0.064516,0.053763,0.096774,0.064516,0.064516,0.053763,0.064516,0.064516,0.053763,0.053763,0.053763,0.053763,0.053763,0.096774,0.053763,0.064516,0.053763,0.053763,0.053763,0.053763,0.053763,0.053763,0.053763,0.053763,0.096774,0.043011,0.043011,0.053763,0.053763,0.053763,0.086022,0.096774,0.376344,0.494624,0.494624,0.096774,0.494624,0.494624,0.494624,0.494624,0.494624,0.494624,0.494624,0.494624,0.494624,0.494624
1,0.088983,0.062500,0.050847,0.096398,0.118644,0.099576,0.127119,0.086864,0.062500,0.093220,0.074153,0.069915,0.104873,0.148305,0.104873,0.039195,0.051907,0.128178,0.048729,0.045551,0.195975,0.083686,0.110169,0.123941,0.066737,0.100636,0.073093,0.059322,0.054025,0.059322,0.050847,0.034958,0.052966,0.059322,0.070975,0.065678,0.083686,0.077331,0.066737,0.082627,0.063559,0.045551,0.050847,0.072034,0.092161,0.077331,0.075212,0.050847,0.034958,0.036017,0.038136,0.069915,0.041314,0.091102,0.069915,0.047669,0.154661,0.023305,0.047669,0.070975,0.069915,0.080508,0.055085,0.086864,0.072034,0.028602,0.090042,0.104873,0.065678,0.077331,0.059322,0.067797,0.046610,0.029661,0.059322,0.056144,0.050847,0.050847,0.077331,0.029661,0.038136,0.022246,0.037076,0.070975,0.045551,0.100636,0.085805,0.025424,0.013771,0.069915,0.044492,0.106992,0.079449,0.078390,0.057203,0.072034,0.127119,0.069915,0.046610,0.052966
2,0.090301,0.081382,0.080268,0.090301,0.095875,0.093645,0.102564,0.095875,0.085842,0.089186,0.085842,0.076923,0.094760,0.092531,0.086957,0.071349,0.069119,0.085842,0.066890,0.063545,0.098105,0.089186,0.098105,0.100334,0.086957,0.093645,0.088071,0.081382,0.073579,0.075808,0.069119,0.055741,0.055741,0.056856,0.089186,0.060201,0.069119,0.074693,0.070234,0.078038,0.071349,0.063545,0.062430,0.064660,0.071349,0.088071,0.070234,0.064660,0.047938,0.047938,0.045708,0.059086,0.053512,0.070234,0.068004,0.061315,0.102564,0.051282,0.053512,0.059086,0.056856,0.063545,0.060201,0.068004,0.064660,0.049052,0.060201,0.099220,0.059086,0.066890,0.063545,0.061315,0.055741,0.049052,0.051282,0.052397,0.051282,0.050167,0.095875,0.042363,0.042363,0.035674,0.039019,0.051282,0.043478,0.055741,0.055741,0.045708,0.037904,0.089186,0.040134,0.059086,0.060201,0.059086,0.055741,0.055741,0.074693,0.066890,0.060201,0.059086
3,0.102207,0.097561,0.092915,0.096400,0.097561,0.096400,0.099884,0.097561,0.092915,0.094077,0.091754,0.087108,0.101045,0.091754,0.089431,0.083624,0.081301,0.087108,0.078978,0.076655,0.087108,0.084785,0.088269,0.103368,0.084785,0.087108,0.085947,0.083624,0.080139,0.081301,0.077816,0.072009,0.070848,0.069686,0.098722,0.069686,0.070848,0.073171,0.072009,0.074332,0.072009,0.069686,0.067364,0.067364,0.069686,0.097561,0.068525,0.066202,0.059233,0.058072,0.055749,0.060395,0.058072,0.063879,0.062718,0.060395,0.102207,0.056911,0.056911,0.059233,0.058072,0.059233,0.058072,0.060395,0.059233,0.052265,0.056911,0.101045,0.056911,0.059233,0.058072,0.056911,0.055749,0.052265,0.052265,0.052265,0.052265,0.052265,0.099884,0.049942,0.048780,0.045296,0.045296,0.049942,0.045296,0.049942,0.049942,0.046458,0.042973,0.096400,0.042973,0.049942,0.049942,0.049942,0.048780,0.048780,0.055749,0.055749,0.052265,0.052265
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

Since we did not provide column names, they are just numbers. And a new id field is used to index data.

In [42]:
print(df.columns)
print(df.index.os_index_field)

Index(['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5',
       '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63',
       '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77',
       '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90',
       '91', '92', '93', '94', '95', '96', '97', '98', '99'],
      dtype='object')
_id


In [43]:
print(df.os_info())

os_index_pattern: smd
Index:
 os_index_field: _id
 is_source_field: False
Mappings:
 capabilities:
   os_field_name  is_source os_dtype os_date_format pd_dtype  is_searchable  is_aggregatable  is_scripted aggregatable_os_field_name
0              0       True   double           None  float64           True             True        False                          0
1              1       True   double           None  float64           True             True        False                          1
10            10       True   double           None  float64           True             True        False                         10
11            11       True   double           None  float64           True             True        False                         11
12            12       True   double           None  float64           True             True        False                         12
13            13       True   double           None  float64           True             True        Fal

## Step 3: Metrics correlation


In [ ]:
 df.to_
